In [5]:
import pandas as pd

test_csv_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKa_basicity_data\gnn_cv\test.csv'

df_test = pd.read_csv(test_csv_path, index_col=0)

In [8]:
SMILES = df_test['Smiles'].tolist()

In [21]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

def calculate_metrics(true_values, pred_values):
    mse = round(mean_squared_error(true_values, pred_values),3)
    mae = round(mean_absolute_error(true_values, pred_values),3)
    r_score = round(r2_score(true_values, pred_values),3)

    return {"mse": mse,
            "mae": mae,
            "r^2": r_score,}

pred_pKa = [3.461677560806274,
 2.5881527423858643,
 4.142161512374878,
 4.086469612121582,
 4.392699527740478,
 4.570784664154052,
 8.319873523712157,
 8.77083963394165,
 9.65727912902832,
 7.989999370574951,
 7.061788959503174,
 10.35153003692627,
 9.162856369018554,
 8.190918769836426,
 6.678215637207031,
 10.451743888854981,
 10.010702629089355,
 10.93015785217285,
 6.292105846405029,
 10.882271881103515,
 8.586725559234619,
 8.081362323760986,
 7.354726142883301,
 11.29662769317627,
 7.010070400238037]
true_pka = df_test['pKa'].tolist()

calculate_metrics(true_values=true_pka,
                  pred_values=pred_pKa)

{'mse': 0.335, 'mae': 0.479, 'r^2': 0.944}

In [18]:
updated_pred_values = []

for index in range(len(pred_pKa)):
    print(f"True: {true_pka[index]}, pred: {pred_pKa[index]}")
    updated_pred_values.append(pred_pKa[index] - 0.2)

True: 3.55, pred: 3.461677560806274
True: 3.51, pred: 2.5881527423858643
True: 4.02, pred: 4.142161512374878
True: 4.35, pred: 4.086469612121582
True: 4.18, pred: 4.392699527740478
True: 4.61, pred: 4.570784664154052
True: 9.12, pred: 8.319873523712157
True: 9.58, pred: 8.77083963394165
True: 8.76, pred: 9.65727912902832
True: 8.1, pred: 7.989999370574951
True: 7.54, pred: 7.061788959503174
True: 9.86, pred: 10.35153003692627
True: 9.57, pred: 9.162856369018554
True: 9.01, pred: 8.190918769836426
True: 7.59, pred: 6.678215637207031
True: 10.36, pred: 10.451743888854981
True: 10.21, pred: 10.010702629089355
True: 10.5, pred: 10.93015785217285
True: 7.11, pred: 6.292105846405029
True: 11.28, pred: 10.882271881103515
True: 9.55, pred: 8.586725559234619
True: 8.97, pred: 8.081362323760986
True: 6.79, pred: 7.354726142883301
True: 11.23, pred: 11.29662769317627
True: 7.2, pred: 7.010070400238037


In [19]:
calculate_metrics(true_values=true_pka,
                  pred_values=updated_pred_values)

{'mse': 0.475, 'mae': 0.577, 'r^2': 0.921}

In [ ]:
from pkasolver.query import QueryModel
from pkasolver.ml_architecture import GINPairV1
import pickle 
import pkasolver
import torch
from os import path
from rdkit import Chem
from pkasolver.query import calculate_microstate_pka_values, draw_pka_reactions
from IPython.display import display

base_path = path.dirname(pkasolver.__file__)
# get input
print("🔮 Predicting...")

SMILES = ['CCC(F)(F)CC(O)=O',
 'OC(C1C2(CC2)C1(F)F)=O',
 'FC1(F)CC(C(O)=O)C1',
 'O=C(O)C(C1)CC21CC(F)(F)C2',
 'FC1(F)CC(C(O)=O)CCC1',
 'OC(C1C2(CC2)C1)=O',
 'FC1(F)CC(N)CCC1',
 'NC(C1)CC21CC(F)(F)C2',
 'NC1CC1',
 'NC1(CF)CCC1',
 'FC1(F)C(N)CCC1',
 'NC1CCC1',
 'FC1(F)CCC(N)CCC1',
 'FC(F)(F)C1CNCCC1',
 'CCC(F)(F)CN',
 'NCCCC(F)([H])[H]',
 'NC(C1)CC21CCC2',
 'CCCCN',
 'F[C@@H]1CNC[C@H]1F',
 'CC1(NC2)CC2C1',
 'FC([H])(F)C1CNCCC1',
 'FC(F)(F)C1CNCC1',
 'FC1(F)CNCCC1',
 'C1(NC2)CC2C1',
 'NCC([H])(F)F']

pred_pka_solver = []

for smile in SMILES:
    print(smile)
    mol = Chem.MolFromSmiles(smile)

    protonation_states = calculate_microstate_pka_values(mol, only_dimorphite=False)

    for i in range(len(protonation_states)):
        state = protonation_states[i]
        print(
            Chem.MolToSmiles(state.protonated_mol),
            Chem.MolToSmiles(state.deprotonated_mol),
        )
        print(state.pka)
        print("😀" + "--"*10 + "😀")

        pred_pka_solver.append(state.pka)